In [ ]:
import os 
os.chdir('drive/My Drive/Urdu2')

In [ ]:
os.chdir('nmt-keras')

In [ ]:
!pip install -e .


Obtaining file:///content/drive/My%20Drive/Urdu2/nmt-keras
  Found existing installation: nmt-keras 0.6
    Can't uninstall 'nmt-keras'. No files were found to uninstall.
  Running setup.py develop for nmt-keras


In [ ]:
from keras_wrapper.dataset import Dataset, saveDataset
from data_engine.prepare_data import keep_n_captions


In [ ]:
ds = Dataset('task', 'task', silence=False)

In [ ]:
ds.setOutput('examples/task/train.en',
             'train',
             type='text',
             id='target_text',
             tokenization='tokenize_none',
             build_vocabulary=True,
             pad_on_batch=True,
             sample_weights=True,
             max_text_len=50,
             max_words=30000,
             min_occ=0)

ds.setOutput('examples/task/dev.en',
             'val',
             type='text',
             id='target_text',
             pad_on_batch=True,
             tokenization='tokenize_none',
             sample_weights=True,
             max_text_len=50,
             max_words=0)

[23/06/2020 17:22:16] 	Applying tokenization function: "tokenize_none".
[23/06/2020 17:22:16] Creating vocabulary for data with data_id 'target_text'.
[23/06/2020 17:22:16] 	 Total: 6284 unique words in 7400 sentences with a total of 192565 words.
[23/06/2020 17:22:16] Creating dictionary of 30000 most common words, covering 100.0% of the text.
[23/06/2020 17:22:16] Loaded "train" set outputs of data_type "text" with data_id "target_text" and length 7400.
[23/06/2020 17:22:16] 	Applying tokenization function: "tokenize_none".
[23/06/2020 17:22:16] Loaded "val" set outputs of data_type "text" with data_id "target_text" and length 300.


In [ ]:
ds.setInput('examples/task/train.ur',
            'train',
            type='text',
            id='source_text',
            pad_on_batch=True,
            tokenization='tokenize_none',
            build_vocabulary=True,
            fill='end',
            max_text_len=50,
            max_words=30000,
            min_occ=0)
ds.setInput('examples/task/dev.ur',
            'val',
            type='text',
            id='source_text',
            pad_on_batch=True,
            tokenization='tokenize_none',
            fill='end',
            max_text_len=50,
            min_occ=0)

[23/06/2020 17:22:19] 	Applying tokenization function: "tokenize_none".
[23/06/2020 17:22:19] Creating vocabulary for data with data_id 'source_text'.
[23/06/2020 17:22:19] 	 Total: 6877 unique words in 7400 sentences with a total of 186176 words.
[23/06/2020 17:22:19] Creating dictionary of 30000 most common words, covering 100.0% of the text.
[23/06/2020 17:22:19] Loaded "train" set inputs of data_type "text" with data_id "source_text" and length 7400.
[23/06/2020 17:22:19] 	Applying tokenization function: "tokenize_none".
[23/06/2020 17:22:19] Loaded "val" set inputs of data_type "text" with data_id "source_text" and length 300.


In [ ]:
ds.setInput('examples/task/train.en',
            'train',
            type='text',
            id='state_below',
            required=False,
            tokenization='tokenize_none',
            pad_on_batch=True,
            build_vocabulary='target_text',
            offset=1,
            fill='end',
            max_text_len=50,
            max_words=30000)
ds.setInput(None,
            'val',
            type='ghost',
            id='state_below',
            required=False)

[23/06/2020 17:22:41] 	Applying tokenization function: "tokenize_none".
[23/06/2020 17:22:41] 	Reusing vocabulary named "target_text" for data with data_id "state_below".
[23/06/2020 17:22:41] Loaded "train" set inputs of data_type "text" with data_id "state_below" and length 7400.
[23/06/2020 17:22:41] Loaded "val" set inputs of data_type "ghost" with data_id "state_below" and length 300.


In [ ]:
  for split, input_text_filename in zip(['train', 'val'], ['examples/task/train.ur', 'examples/task/dev.ur']):
    ds.setRawInput(input_text_filename,
                  split,
                  type='file-name',
                  id='raw_source_text',
                  overwrite_split=True)

[23/06/2020 17:23:12] Loaded "train" set inputs of type "file-name" with id "raw_source_text".
[23/06/2020 17:23:12] Loaded "val" set inputs of type "file-name" with id "raw_source_text".


In [ ]:
keep_n_captions(ds, repeat=1, n=1, set_names=['val'])


[23/06/2020 17:23:21] Keeping 1 captions per input on the val set.
[23/06/2020 17:23:21] Samples reduced to 300 in val set.


In [ ]:
saveDataset(ds, 'datasets')


[23/06/2020 17:23:27] <<< creating directory datasets ... >>>
[23/06/2020 17:23:27] <<< Saving Dataset instance to datasets/Dataset_task.pkl ... >>>
[23/06/2020 17:23:27] <<< Dataset instance saved >>>


In [ ]:
!pip install git+https://github.com/MarcBS/keras.git


  Cloning https://github.com/MarcBS/keras.git to /tmp/pip-req-build-56lht_5o
  Running command git clone -q https://github.com/MarcBS/keras.git /tmp/pip-req-build-56lht_5o
  Created wheel for Keras: filename=Keras-2.3.1.1-cp36-none-any.whl size=487500 sha256=52274eab082cfffe7fd6117e8f4ff02070f18beb56e15da3733d928e59cfc139
  Stored in directory: /tmp/pip-ephem-wheel-cache-zz9ymiz6/wheels/83/59/60/d798700bbebbe19766125ac1b5cd87da0dc780bf6b11acecf6
Successfully built Keras
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [ ]:
from config import load_parameters
from nmt_keras.model_zoo import TranslationModel
from keras_wrapper.cnn_model import loadModel
from keras_wrapper.dataset import loadDataset
from keras_wrapper.extra.callbacks import PrintPerformanceMetricOnEpochEndOrEachNUpdates
params = load_parameters()


Using TensorFlow backend.
[24/06/2020 05:36:27] NumExpr defaulting to 2 threads.
[24/06/2020 05:36:27] <<< Cupy not available. Using numpy. >>>


In [ ]:
dataset = loadDataset('datasets/Dataset_task.pkl')


[24/06/2020 05:44:52] <<< Loading Dataset instance from datasets/Dataset_task.pkl ... >>>
[24/06/2020 05:44:52] <<< Dataset instance loaded >>>


In [ ]:
params['INPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['source_text']
params['OUTPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['target_text']

In [ ]:
params['MODEL_TYPE'] = 'AttentionRNNEncoderDecoder' #  Supported models: 'AttentionRNNEncoderDecoder' and 'Transformer'.
nmt_model = TranslationModel(params,
                             model_type=params['MODEL_TYPE'], 
                             model_name='tutorial_model',
                             vocabularies=dataset.vocabulary,
                             store_path='trained_models/tutorial_model/',
                             verbose=True)


[23/06/2020 17:24:13] <<< Building AttentionRNNEncoderDecoder Translation_Model >>>


[23/06/2020 17:24:13] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:650: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



[23/06/2020 17:24:13] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4786: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



[23/06/2020 17:24:13] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:157: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[23/06/2020 17:24:14] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3561: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


-----------------------------------------------------------------------------------
		TranslationModel instance
-----------------------------------------------------------------------------------
_model_type: AttentionRNNEncoderDecoder
name: tutorial_model
model_path: trained_models/tutorial_model/
verbose: True

Params:
	ACCUMULATE_GRADIENTS: 1
	ADDITIONAL_OUTPUT_MERGE_MODE: Add
	ALIGN_FROM_RAW: True
	ALPHA_FACTOR: 0.6
	AMSGRAD: False
	APPLY_DETOKENIZATION: False
	ATTENTION_DROPOUT_P: 0.0
	ATTENTION_MODE: add
	ATTENTION_SIZE: 32
	BATCH_NORMALIZATION_MODE: 1
	BATCH_SIZE: 50
	BEAM_SEARCH: True
	BEAM_SIZE: 6
	BETA_1: 0.9
	BETA_2: 0.999
	BIDIRECTIONAL_DEEP_ENCODER: True
	BIDIRECTIONAL_ENCODER: True
	BIDIRECTIONAL_MERGE_MODE: concat
	BPE_CODES_PATH: examples/EuTrans//training_codes.joint
	CLASSIFIER_ACTIVATION: softmax
	CLIP_C: 5.0
	CLIP_V: 0.0
	COVERAGE_NORM_FACTOR: 0.2
	COVERAGE_PENALTY: False
	DATASET_NAME: EuTrans
	DATASET_STORE_PATH: datasets/
	DATA_AUGMENTATION: False
	DATA_ROOT_PATH

[23/06/2020 17:24:15] From /content/drive/My Drive/Urdu2/nmt-keras/nmt_keras/model_zoo.py:213: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

[23/06/2020 17:24:15] Preparing optimizer and compiling. Optimizer configuration: 
	 LR: 0.001
	 LOSS: categorical_crossentropy
	 BETA_1: 0.9
	 BETA_2: 0.999
	 EPSILON: 1e-08


[23/06/2020 17:24:15] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1192: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



In [ ]:

inputMapping = dict()
for i, id_in in enumerate(params['INPUTS_IDS_DATASET']):
    pos_source = dataset.ids_inputs.index(id_in)
    id_dest = nmt_model.ids_inputs[i]
    inputMapping[id_dest] = pos_source
nmt_model.setInputsMapping(inputMapping)

outputMapping = dict()
for i, id_out in enumerate(params['OUTPUTS_IDS_DATASET']):
    pos_target = dataset.ids_outputs.index(id_out)
    id_dest = nmt_model.ids_outputs[i]
    outputMapping[id_dest] = pos_target
nmt_model.setOutputsMapping(outputMapping)


In [ ]:
is_transformer = params.get('ATTEND_ON_OUTPUT', 'transformer' in params['MODEL_TYPE'].lower())
search_params = {
    'language': 'en',
    'tokenize_f': eval('dataset.' + 'tokenize_none'),
    'beam_size': 12,
    'optimized_search': True,
    'model_inputs': params['INPUTS_IDS_MODEL'],
    'model_outputs': params['OUTPUTS_IDS_MODEL'],
    'dataset_inputs':  params['INPUTS_IDS_DATASET'],
    'dataset_outputs':  params['OUTPUTS_IDS_DATASET'],
    'n_parallel_loaders': 1,
    'maxlen': 50,
    'normalize_probs': True,
    'pos_unk': True and not is_transformer,  # Pos_unk is unimplemented for transformer models
    'heuristic': 0,
    'state_below_maxlen': -1,
    'attend_on_output': is_transformer,
    'val': {'references': dataset.extra_variables['val']['target_text']}
  }

vocab = dataset.vocabulary['target_text']['idx2words']
callbacks = []
input_text_id = params['INPUTS_IDS_DATASET'][0]

callbacks.append(PrintPerformanceMetricOnEpochEndOrEachNUpdates(nmt_model,
                                                                dataset,
                                                                gt_id='target_text',
                                                                metric_name=['sacrebleu'],
                                                                set_name=['val'],
                                                                batch_size=50,
                                                                each_n_epochs=1,
                                                                extra_vars=search_params,
                                                                reload_epoch=0,
                                                                is_text=True,
                                                                input_text_id=input_text_id,
                                                                index2word_y=vocab,
                                                                sampling_type='max_likelihood',
                                                                beam_search=True,
                                                                save_path=nmt_model.model_path,
                                                                start_eval_on_epoch=0,
                                                                write_samples=True,
                                                                write_type='list',
                                                                verbose=True))

In [ ]:
training_params = {'n_epochs': 100,
                   'batch_size': 50,
                   'maxlen': 50,
                   'epochs_for_save': 1,
                   'verbose': 1,
                   'eval_on_sets': [], 
                   'n_parallel_loaders': 1,
                   'extra_callbacks': callbacks,
                   'reload_epoch': 0,
                   'epoch_offset': 0}


In [ ]:
nmt_model.trainNet(dataset, training_params)


[23/06/2020 18:11:23] <<< Training model >>>
[23/06/2020 18:11:23] Training parameters: { 
	batch_size: 50
	class_weights: None
	da_enhance_list: []
	da_patch_type: resize_and_rndcrop
	data_augmentation: False
	each_n_epochs: 1
	epoch_offset: 0
	epochs_for_save: 1
	eval_on_epochs: True
	eval_on_sets: []
	extra_callbacks: [<keras_wrapper.extra.callbacks.EvalPerformance object at 0x7fb0981e93c8>]
	homogeneous_batches: False
	initial_lr: 1.0
	joint_batches: 4
	lr_decay: None
	lr_gamma: 0.1
	lr_half_life: 50000
	lr_reducer_exp_base: 0.5
	lr_reducer_type: linear
	lr_warmup_exp: -1.5
	maxlen: 50
	mean_substraction: False
	metric_check: None
	min_delta: 0.0
	min_lr: 1e-09
	n_epochs: 100
	n_gpus: 1
	n_parallel_loaders: 1
	normalization_type: None
	normalize: False
	num_iterations_val: None
	patience: 0
	patience_check_split: val
	reduce_each_epochs: True
	reload_epoch: 0
	shuffle: True
	start_eval_on_epoch: 0
	start_reduction_on_epoch: 0
	tensorboard: False
	tensorboard_params: {'log_dir': 'te

Epoch 1/100
148/148 [==============================] - 99s 669ms/step - loss: 1.6462 - perplexity: 1611.0706


[23/06/2020 18:13:02] <<< Saving model to trained_models/tutorial_model/epoch_1 ... >>>


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '
[23/06/2020 18:13:05] <<< Model saved >>>

[23/06/2020 18:13:05] <<< Predicting outputs of val set >>>




 Total cost: 11879.292215 	 Average cost: 39.597641
The sampling took: 58.747998 secs (Speed: 0.195827 sec/sample)


...

[23/06/2020 22:29:36] <<< Saving model to trained_models/tutorial_model/epoch_100 ... >>>


[23/06/2020 22:29:36] <<< Model saved >>>

[23/06/2020 22:29:36] <<< Predicting outputs of val set >>>




 Total cost: 5478.476755 	 Average cost: 18.261589
The sampling took: 52.739933 secs (Speed: 0.175800 sec/sample)


[23/06/2020 22:30:29] Prediction output 0: target_text (text)
[23/06/2020 22:30:29] Decoding beam search prediction ...
[23/06/2020 22:30:29] Using heuristic 0
[23/06/2020 22:30:29] Evaluating on metric sacrebleu
[23/06/2020 22:30:29] Computing SacreBleu scores on the val split...
[23/06/2020 22:30:29] Bleu_4: 7.996934813441331
[23/06/2020 22:30:29] Done evaluating on metric sacrebleu


[23/06/2020 22:30:29] 
<<< Progress plot saved in trained_models/tutorial_model/epoch_100.jpg >>>
[23/06/2020 22:30:29] <<< Finished training model >>>


In [ ]:
dataset.setInput('examples/task/test.ur',
            'test',
            type='text',
            id='source_text',
            pad_on_batch=True,
            tokenization='tokenize_none',
            fill='end',
            max_text_len=50,
            min_occ=0)

dataset.setInput(None,
            'test',
            type='ghost',
            id='state_below',
            required=False)

dataset.setRawInput('examples/task/test.ur',
              'test',
              type='file-name',
              id='raw_source_text',
              overwrite_split=True)


[24/06/2020 05:44:59] 	Applying tokenization function: "tokenize_none".
[24/06/2020 05:44:59] Loaded "test" set inputs of data_type "text" with data_id "source_text" and length 257.
[24/06/2020 05:44:59] Loaded "test" set inputs of data_type "ghost" with data_id "state_below" and length 257.
[24/06/2020 05:44:59] Loaded "test" set inputs of type "file-name" with id "raw_source_text".


In [ ]:
params['INPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len[params['INPUTS_IDS_DATASET'][0]]
params['OUTPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len[params['OUTPUTS_IDS_DATASET'][0]]

# Load model
nmt_model = loadModel('trained_models/tutorial_model', 33)


[24/06/2020 05:57:36] <<< Loading model from trained_models/tutorial_model/epoch_33_Model_Wrapper.pkl ... >>>
[24/06/2020 05:57:36] <<< Loading model from trained_models/tutorial_model/epoch_33.h5 ... >>>
[24/06/2020 05:57:40] <<< Loading optimized model... >>>
[24/06/2020 05:57:47] <<< Optimized model loaded. >>>
[24/06/2020 05:57:47] <<< Model loaded in 10.347 seconds. >>>


In [ ]:
is_transformer = params.get('ATTEND_ON_OUTPUT', 'transformer' in params['MODEL_TYPE'].lower())

params_prediction = {
    'language': 'en',
    'tokenize_f': eval('dataset.' + 'tokenize_none'),
    'beam_size': 12,
    'optimized_search': True,
    'model_inputs': params['INPUTS_IDS_MODEL'],
    'model_outputs': params['OUTPUTS_IDS_MODEL'],
    'dataset_inputs':  params['INPUTS_IDS_DATASET'],
    'dataset_outputs':  params['OUTPUTS_IDS_DATASET'],
    'n_parallel_loaders': 1,
    'maxlen': 50,
    'normalize_probs': True,
    'pos_unk': True and not is_transformer,
    'heuristic': 0,
    'state_below_maxlen': -1,
    'predict_on_sets': ['test'],
    'verbose': 0,
    'attend_on_output': is_transformer
  }
predictions = nmt_model.predictBeamSearchNet(dataset, params_prediction)['test']


[24/06/2020 05:58:39] <<< Predicting outputs of test set >>>




 Total cost: 7308.658398 	 Average cost: 28.438360
The sampling took: 139.079838 secs (Speed: 0.541167 sec/sample)


In [ ]:
from keras_wrapper.utils import decode_predictions_beam_search
vocab = dataset.vocabulary['target_text']['idx2words']
samples = predictions['samples'] # Get word indices from the samples.

predictions = decode_predictions_beam_search(samples,  
                                             vocab,
                                             verbose=params['VERBOSE'])

[24/06/2020 06:01:05] Decoding beam search prediction ...


In [ ]:
filepath = 'test.pred'
from keras_wrapper.extra.read_write import list2file
list2file(filepath, predictions)
!head -n 4 test.pred

Neither entered into the way into the people , but took up a great way into the way to the poor ; and not as he had been a season , as they ought to ask that he written , saying , It is a season , lest they
There shall be the days of those days ; but they shall not see : and they enter into their own : and they persecute us into death ; and he walked against them .
And cast them out of the fame of them , and they cast hold for their garments , and they have taken up a crown of their souls . And when they were an hungred , and cut about their own words .
That there were afar off , and an hair , and ten , and judgeth according to the same , and took the same things .


In [ ]:
dataset.setOutput('examples/task/test.en',
             'test',
             type='text',
             id='target_text',
             pad_on_batch=True,
             tokenization='tokenize_none',
             sample_weights=True,
             max_text_len=50,
             max_words=0)


Exception: ignored

In [ ]:
keep_n_captions(dataset, repeat=1, n=1, set_names=['test'])

[24/06/2020 06:01:17] Keeping 1 captions per input on the test set.
[24/06/2020 06:01:17] Samples reduced to 257 in test set.


In [ ]:

from keras_wrapper.extra.evaluation import select
metric = 'sacrebleu'
# Apply sampling
extra_vars = dict()
extra_vars['tokenize_f'] = eval('dataset.' + 'tokenize_none')
extra_vars['language'] = params['TRG_LAN']
extra_vars['test'] = dict()
extra_vars['test']['references'] = dataset.extra_variables['test']['target_text']
metrics = select[metric](pred_list=predictions,
                                          verbose=1,
                                          extra_vars=extra_vars,
                                          split='test')

[24/06/2020 06:01:18] Computing SacreBleu scores on the test split...
[24/06/2020 06:01:18] Bleu_4: 7.300801982565925
